In [1]:
import openai
openai.api_key = "api_key"
code2lang = {
    'ar':       'Arabic',
    'cs':       'Czech',
    'da':       'Danish',
    'de':       'German',
    'el':       'Greek',
    'en':       'English',
    'es':       'Spanish',
    'fi':       'Finnish',
    'fr':       'French',
    'he':       'Hebrew',
    'hi':       'Hindi',
    'hu':       'Hungarian',
    'id':       'Indonesian',
    'it':       'Italian',
    'ja':       'Japanese',
    'ko':       'Korean',
    'nb':       'Norwegian Bokmål',
    'nl':       'Dutch',
    'pl':       'Polish',
    'pt':       'Portuguese',
    'ro':       'Romanian',
    'ru':       'Russian',
    'sk':       'Slovak',
    'sv':       'Swedish',
    'th':       'Thai',
    'tr':       'Turkish',
    'uk':       'Ukrainian',
    'vi':       'Vietnamese',
    'zh-Hans':  'Simplified Chinese',
    'zh-Hant':  'Traditional Chinese',
    #'zh-chs':  'Simplified Chinese', # drop
    #'zh-cht':  'Traditional Chinese', # drops
}




In [2]:
from openai import OpenAI

api_key = "api_key"
client = OpenAI(api_key = api_key)

# assuming code2lang is defined elsewhere
# code2lang = {"en": "English", "es": "Spanish", ...}

def get_prompt(target_code, content, source_code):
    target_lang = code2lang[target_code]
    source_lang = code2lang[source_code]
    return f"translate the text in {source_lang} language to {target_lang} language. text: {content}. translated text: "

def callchatgpt_v1(target_code, content, source_code):
    prompt = get_prompt(target_code, content, source_code)

    messages = [
        {"role": "system", "content": "your job is to translate text according to requirements, only return translated text back."},
        {"role": "user", "content": prompt},
    ]

    r = client.chat.completions.create(
        model="gpt-4.1",
        messages=messages,
        temperature=0,
        #max_tokens=250
    )
    return r.choices[0].message.content.strip()


import time

def callchatgpt(target_code, content, source_code):
    prompt = get_prompt(target_code, content, source_code)

    messages = [
        {"role": "system", 
         "content": 
         """We are translating texts in our app, which is for users to learn language. 
         Your job is to translate text according to requirements, only return translated text back.
         Pay attention to punctuation in the end, see when to include it and when not to include it.
         In general, for short text, no punctuation is needed.
         """
         },
        {"role": "user", "content": prompt},
    ]
    attempts = 0
    while attempts < 3:
        try:
            r = client.chat.completions.create(
                model="gpt-4.1",
                messages=messages,
                temperature=0,
                #max_tokens=250
            )
            return r.choices[0].message.content.strip()
        except Exception as e:
            attempts += 1
            if attempts == 3:
                raise
            time.sleep(1)  # Wait a second before retrying (optional)

In [3]:
callchatgpt("zh-Hans","apple","en")

'苹果'

In [4]:
import json

# 1. Load the JSON
with open('Localizable 7.xcstrings', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 2. Collect all language codes
langs = set()
for entry in data.get('strings', {}).values():
    for lang_code in entry.get('localizations', {}):
        langs.add(lang_code)

# 3. Output
print(f"Found {len(langs)} unique language codes:")
print(sorted(langs))

Found 30 unique language codes:
['ar', 'cs', 'da', 'de', 'el', 'en', 'es', 'fi', 'fr', 'he', 'hi', 'hu', 'id', 'it', 'ja', 'ko', 'nb', 'nl', 'pl', 'pt', 'ro', 'ru', 'sk', 'sv', 'th', 'tr', 'uk', 'vi', 'zh-Hans', 'zh-Hant']


In [10]:
import json

# --- 1. Load the JSON ---
with open('Localizable 7 updated.json', 'r', encoding='utf-8') as f:
#with open('translations_filled.json', 'r', encoding='utf-8') as f:

    data = json.load(f)

# --- 2. Build the global set of all languages ---
all_langs = set()
for entry in data.get('strings', {}).values():
    all_langs.update(entry.get('localizations', {}).keys())

# --- 3. Scan each string entry for missing or non-translated langs ---
report = {}

simple_key_value = {}

for key, entry in data.get('strings', {}).items():
    locs = entry.get('localizations', {})
    present = set(locs.keys())
    missing = sorted(all_langs - present)

    
    

    
    # find langs whose state != "translated"
    bad_states = []
    for lang, info in locs.items():
        state = info.get('stringUnit', {}).get('state')
        this_value = info.get('stringUnit', {}).get('value')
        if state != 'translated':
            bad_states.append((lang, state))

    if missing or bad_states:
        report[key] = {
            'missing_languages': missing,
            'non_translated': bad_states
        }
        
    # store simple key and value like smile
    if len(key)<=10:
        simple_key_value[key] = this_value

# --- 4. Print a summary ---
for key, issues in report.items():
    print(f"String key: {key!r}")
    if issues['missing_languages']:
        print("   Missing langs:", issues['missing_languages'])
    if issues['non_translated']:
        print("   Non-translated states:")
        for lang, state in issues['non_translated']:
            print(f"      • {lang}: {state}")
    print()

# Optionally, print a final tally
print(f"Checked {len(data.get('strings',{}))} strings.")
print(f"{len(report)} of them have missing or non-translated entries.")

String key: '1st Grade'
   Missing langs: ['en']

String key: '2nd Grade'
   Missing langs: ['en']

String key: '3rd Grade'
   Missing langs: ['en']

String key: '4th Grade'
   Missing langs: ['en']

String key: '5th Grade'
   Missing langs: ['en']

String key: '6th Grade'
   Missing langs: ['en']

String key: '7th Grade'
   Missing langs: ['en']

String key: '8th Grade'
   Missing langs: ['en']

String key: '9th Grade'
   Missing langs: ['en']

String key: '10th Grade'
   Missing langs: ['en']

String key: '11th Grade'
   Missing langs: ['en']

String key: '12th Grade'
   Missing langs: ['en']

String key: 'Add a New Langauge'
   Missing langs: ['en']

String key: 'Add attributes to help you organize'
   Missing langs: ['en']

String key: 'Add Image'
   Missing langs: ['en']

String key: 'Add tags to help you organize'
   Missing langs: ['en']

String key: 'Add your favorite video to GoLingo and start learning languages now!'
   Missing langs: ['en']

String key: 'Add/Delete Image'
  

In [6]:
simple_key_value={
    '':'',
    '≡':"≡",
    '😀':'😀',
    '?':'?'
}

In [7]:
simple_key_value[""]

''

In [8]:
import json
import time


# 1) load the original file
with open('Localizable 7.xcstrings', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 2) define your full set of languages (minus 'en')
all_langs = ['ar', 'cs', 'da', 'de', 'el', 'es', 'fi', 'fr', 'he', 'hi', 'hu', 
 'id', 'it', 'ja', 'ko', 'nb', 'nl', 'pl', 'pt', 'ro', 'ru', 'sk', 
 'sv', 'th', 'tr', 'uk', 'vi', 'zh-Hans', 'zh-Hant']



# 3) iterate strings
for key, entry in data.get('strings', {}).items():

    src = key       # your source English text
    locs = entry.setdefault('localizations', {})

    

    # remove zh-chs and zh-cht, not needed
    for lang in ['zh-chs', 'zh-cht']:
        if lang in locs:
            print("delete zh-chs or zh-cht")
            del locs[lang]
    
    # ensure every lang-code is present
    for lang in all_langs:
        if lang not in locs:
            locs[lang] = {
                'stringUnit': {
                    'state': 'translated',
                    'value': ''
                }
            }

    # for simple keys, just use the key
    if src in simple_key_value:
        for lang in all_langs+['en']:

            locs[lang] = {
                'stringUnit': {
                    'state': 'translated',
                    'value': simple_key_value[src]
                }
            }
        continue
    
    
    # if 'en' is inside, use the value of en as the new key src
    for lang, block in locs.items():
        su = block.setdefault('stringUnit', {})
        val   = su.get('value','').strip()
        
        if lang == "en" and len(val)>0:
            src = val
            #change en state to be translated, to be avoided to translated by ai again
            su['state'] = 'translated'
            
            print(su)
            print(f"***************  src changed from {key} to {src}")
    
    
    # now for each language, if state != translated OR value is blank → re-translate
    

    for lang, block in locs.items():
        su = block.setdefault('stringUnit', {})
        state = su.get('state')
        val   = su.get('value','').strip()

        #if state == "new":
        #    su['state'] = 'translated'
        #    continue
            
        
            
        if state != 'translated' or not val:
            print("to do lang")
            print(lang)
            # perform translation
            dest = lang

            tr_text = callchatgpt(lang,src,"en")
            su['value'] = tr_text
            su['state'] = 'translated'
            
            print("***********************")
            print(src)
            print(su)

# 4) write out the filled result
with open('translations_filled.json', 'w', encoding='utf-8') as outf:
    json.dump(data, outf, ensure_ascii=False, indent=2)

print("Done — new file written to translations_filled.json")

{'state': 'translated', 'value': '  %1$lld/%2$lld'}
***************  src changed from   %lld/%lld to %1$lld/%2$lld
{'state': 'translated', 'value': '%@'}
***************  src changed from (%@) to %@
{'state': 'translated', 'value': '%@'}
***************  src changed from %@ to %@
{'state': 'translated', 'value': '%1$@\r\n%2$@'}
***************  src changed from %@
%@ to %1$@
%2$@
{'state': 'translated', 'value': '%1$@: %2$.1f Hz'}
***************  src changed from %@: %.1f Hz to %1$@: %2$.1f Hz
{'state': 'translated', 'value': '%1$@: %2$@'}
***************  src changed from %@: %@ to %1$@: %2$@
{'state': 'translated', 'value': '%lld'}
***************  src changed from %lld to %lld
{'state': 'translated', 'value': '%1$lld %2$@'}
***************  src changed from %lld %@ to %1$lld %2$@
{'state': 'translated', 'value': '%1$lld 😀 out %2$lld 😐'}
***************  src changed from %lld 😀 out %lld 😐 to %1$lld 😀 out %2$lld 😐
{'state': 'translated', 'value': '%1$lld/%2$lld'}
***************  src

***********************
2nd Grade
{'state': 'translated', 'value': 'ป.2'}
to do lang
tr
***********************
2nd Grade
{'state': 'translated', 'value': '2. Sınıf'}
to do lang
uk
***********************
2nd Grade
{'state': 'translated', 'value': '2 клас'}
to do lang
vi
***********************
2nd Grade
{'state': 'translated', 'value': 'Lớp 2'}
to do lang
zh-Hans
***********************
2nd Grade
{'state': 'translated', 'value': '二年级'}
to do lang
zh-Hant
***********************
2nd Grade
{'state': 'translated', 'value': '二年級'}
{'state': 'translated', 'value': '3. Return here to practice and play games.'}
***************  src changed from 3. Return here to practice and play games to 3. Return here to practice and play games.
{'state': 'translated', 'value': '3D Flight Simulation Game'}
***************  src changed from 3D Flight Learning Experience to 3D Flight Simulation Game
to do lang
ar
***********************
3rd Grade
{'state': 'translated', 'value': 'الصف الثالث'}
to do lang
cs


***********************
5th Grade
{'state': 'translated', 'value': '5. ročník'}
to do lang
sv
***********************
5th Grade
{'state': 'translated', 'value': 'Årskurs 5'}
to do lang
th
***********************
5th Grade
{'state': 'translated', 'value': 'ป.5'}
to do lang
tr
***********************
5th Grade
{'state': 'translated', 'value': '5. Sınıf'}
to do lang
uk
***********************
5th Grade
{'state': 'translated', 'value': '5 клас'}
to do lang
vi
***********************
5th Grade
{'state': 'translated', 'value': 'Lớp 5'}
to do lang
zh-Hans
***********************
5th Grade
{'state': 'translated', 'value': '五年级'}
to do lang
zh-Hant
***********************
5th Grade
{'state': 'translated', 'value': '五年級'}
{'state': 'translated', 'value': '6 Months Access'}
***************  src changed from 6 Months Access to 6 Months Access
to do lang
ar
***********************
6th Grade
{'state': 'translated', 'value': 'الصف السادس'}
to do lang
cs
***********************
6th Grade
{'state': 'tr

***********************
8th Grade
{'state': 'translated', 'value': '8. ročník'}
to do lang
sv
***********************
8th Grade
{'state': 'translated', 'value': 'Åttonde klass'}
to do lang
th
***********************
8th Grade
{'state': 'translated', 'value': 'ม.2'}
to do lang
tr
***********************
8th Grade
{'state': 'translated', 'value': '8. Sınıf'}
to do lang
uk
***********************
8th Grade
{'state': 'translated', 'value': '8 клас'}
to do lang
vi
***********************
8th Grade
{'state': 'translated', 'value': 'Lớp 8'}
to do lang
zh-Hans
***********************
8th Grade
{'state': 'translated', 'value': '八年级'}
to do lang
zh-Hant
***********************
8th Grade
{'state': 'translated', 'value': '八年級'}
to do lang
ar
***********************
9th Grade
{'state': 'translated', 'value': 'الصف التاسع'}
to do lang
cs
***********************
9th Grade
{'state': 'translated', 'value': '9. třída'}
to do lang
da
***********************
9th Grade
{'state': 'translated', 'value': '9. 

***********************
11th Grade
{'state': 'translated', 'value': '11. ročník'}
to do lang
sv
***********************
11th Grade
{'state': 'translated', 'value': 'Årskurs 11'}
to do lang
th
***********************
11th Grade
{'state': 'translated', 'value': 'มัธยมศึกษาปีที่ 5'}
to do lang
tr
***********************
11th Grade
{'state': 'translated', 'value': '11. Sınıf'}
to do lang
uk
***********************
11th Grade
{'state': 'translated', 'value': '11 клас'}
to do lang
vi
***********************
11th Grade
{'state': 'translated', 'value': 'Lớp 11'}
to do lang
zh-Hans
***********************
11th Grade
{'state': 'translated', 'value': '十一年级'}
to do lang
zh-Hant
***********************
11th Grade
{'state': 'translated', 'value': '十一年級'}
to do lang
ar
***********************
12th Grade
{'state': 'translated', 'value': 'الصف الثاني عشر'}
to do lang
cs
***********************
12th Grade
{'state': 'translated', 'value': '12. třída'}
to do lang
da
***********************
12th Grade
{'st

...

***********************
Present Progressive Tense
{'state': 'translated', 'value': 'Ενεστώτας διαρκείας'}
to do lang
es
***********************
Present Progressive Tense
{'state': 'translated', 'value': 'Tiempo presente progresivo'}
to do lang
fi
***********************
Present Progressive Tense
{'state': 'translated', 'value': 'Kestopreesens'}
to do lang
fr
***********************
Present Progressive Tense
{'state': 'translated', 'value': 'Présent progressif'}
to do lang
he
***********************
Present Progressive Tense
{'state': 'translated', 'value': 'זמן הווה מתמשך'}
to do lang
hi
***********************
Present Progressive Tense
{'state': 'translated', 'value': 'वर्तमान प्रगतिशील काल'}
to do lang
hu
***********************
Present Progressive Tense
{'state': 'translated', 'value': 'Folyamatos jelen idő'}
to do lang
id
***********************
Present Progressive Tense
{'state': 'translated', 'value': 'Present Progressive Tense'}
to do lang
it
***********************
Present Prog

***********************
Sentence Length: %lld
{'state': 'translated', 'value': 'Délka věty: %lld.'}
to do lang
da
***********************
Sentence Length: %lld
{'state': 'translated', 'value': 'Sætningslængde: %lld'}
to do lang
de
***********************
Sentence Length: %lld
{'state': 'translated', 'value': 'Satzlänge: %lld'}
to do lang
el
***********************
Sentence Length: %lld
{'state': 'translated', 'value': 'Μήκος πρότασης: %lld.'}
to do lang
es
***********************
Sentence Length: %lld
{'state': 'translated', 'value': 'Longitud de la oración: %lld.'}
to do lang
fi
***********************
Sentence Length: %lld
{'state': 'translated', 'value': 'Lauseen pituus: %lld'}
to do lang
fr
***********************
Sentence Length: %lld
{'state': 'translated', 'value': 'Longueur de la phrase : %lld.'}
to do lang
he
***********************
Sentence Length: %lld
{'state': 'translated', 'value': 'אורך המשפט: \u200e%lld'}
to do lang
hi
***********************
Sentence Length: %lld
{'sta

***********************
We hope you're enjoying the speech recognition feature on this page. Exciting new features await our subscribers. To enjoy an ad-free experience, consider subscribing for less than 30 cents a day.
{'state': 'translated', 'value': 'Toivomme, että nautit tämän sivun puheentunnistusominaisuudesta. Jännittäviä uusia ominaisuuksia odottaa tilaajiamme. Jos haluat nauttia mainoksettomasta kokemuksesta, harkitse tilaamista alle 30 sentillä päivässä.'}
{'state': 'translated', 'value': 'Welcome to GoLingo. Let us guide you through some basic basic features to get started.'}
***************  src changed from Welcome to GoLingo. Let us guide you through some basic basic features to get started. to Welcome to GoLingo. Let us guide you through some basic basic features to get started.
{'state': 'translated', 'value': 'Subscribe for just 30¢ a day to unlock advanced tools like AI, high-quality audio, speech recognition, full access to all games and activities, autoplay, backgr

***********************
Word/Phrase repeats after the translation per round?
{'state': 'translated', 'value': 'Слово/фраза повторяется после перевода в каждом раунде?'}
to do lang
sk
***********************
Word/Phrase repeats after the translation per round?
{'state': 'translated', 'value': 'Slovo/fráza sa opakuje po preklade v každom kole?'}
to do lang
sv
***********************
Word/Phrase repeats after the translation per round?
{'state': 'translated', 'value': 'Upprepas ord/frase efter översättningen per omgång?'}
to do lang
th
***********************
Word/Phrase repeats after the translation per round?
{'state': 'translated', 'value': 'คำ/วลีจะซ้ำหลังจากแปลในแต่ละรอบหรือไม่'}
to do lang
tr
***********************
Word/Phrase repeats after the translation per round?
{'state': 'translated', 'value': 'Kelime/İfade her turdan sonra çeviriden sonra tekrar eder mi'}
to do lang
uk
***********************
Word/Phrase repeats after the translation per round?
{'state': 'translated', 'value

Done — new file written to translations_filled.json
